In [ ]:
import numpy as np
import scarlet
import logging
import tensorflow_hub as hub
import tensorflow as tf
logger = logging.getLogger('scarlet')
logger.setLevel(logging.DEBUG)

In [2]:
import proxmin
import proxmin.operators

In [26]:
class PixelCNNConstraint(scarlet.Constraint):
    
    def __init__(self, module, session):
        self.x = tf.placeholder(shape=(1,64,64,1), dtype=tf.float32)
        self.pixelcnn = module(self.x)
        self.session = session

    def prox_pixelcnn(self, X, step):
        print(X.shape)
        print(X)
        # all images in pixelcnn are normalized to unit flux
        X /= X.sum()
        grad = self.session.run(self.pixelcnn, feed_dict={self.x: X})
        print(grad)
        res = X + step*grad[0,:,:,0]
        return res

    def prox_sed(self, shape):
        return proxmin.operators.prox_plus

    def prox_morph(self, shape):
        return self.prox_pixelcnn

In [4]:
module_path='modules/pixelcnn_gradients'
pixelcnn = hub.Module(module_path)
sess= tf.Session()
sess.run(tf.global_variables_initializer())
x = tf.placeholder(shape=(1,64,64,1), dtype=tf.float32)

In [5]:
# quick test of the gradient module
%pylab inline
im = 0.01*randn(1,64,64,1)
im0 = im+0
grads = pixelcnn(x)

Populating the interactive namespace from numpy and matplotlib


In [ ]:
for i in range(1000):
    d = sess.run(grads, feed_dict={x: im})
    im -= 0.001*d/(64.*64)

In [ ]:
# It reproduces more or less the noise correlation learned from training
subplot(121)
imshow(im0[0,:,:,0])
subplot(122)
imshow(im[0,:,:,0])

In [7]:
data = np.load("../scarlet/data/test_sim/data.npz")
img = data["images"]
filters = data["filters"]

# open file and perform detection
B = len(img)

from astropy.table import Table as ApTable
catalog = ApTable.read("../scarlet/data/test_sim/true_catalog.fits")
bg_rms = np.array([1e-3]*len(img))

In [27]:
# run scarlet
config = scarlet.Config(source_sizes=[64], accelerated=True)
sources = [scarlet.PointSource((obj['y'],obj['x']), img, shape=(64,64), normalization=scarlet.Normalization.S, config=config, constraints=PixelCNNConstraint(pixelcnn, sess)) for obj in catalog]
blend = scarlet.Blend(sources).set_data(img, config=config)

In [ ]:
plt.figure()
plt.imshow(sources[0][0].morph)
plt.show()

In [ ]:
len(sources)

In [ ]:
sources[0][0].morph.shape

In [29]:
blend.fit(100, e_rel=1e-4)

(64, 64)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.00086367 ...  0.          0.
   0.        ]
 [ 0.          0.         -0.00013609 ...  0.          0.
   0.        ]
 [ 0.          0.         -0.00049934 ...  0.          0.
   0.        ]]


ValueError: Cannot feed value of shape (64, 64) for Tensor 'Placeholder_36:0', which has shape '(1, 64, 64, 1)'